In [ ]:
!wget   http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2 http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2
!bunzip2 /content/shape_predictor_5_face_landmarks.dat.bz2 /content/dlib_face_recognition_resnet_model_v1.dat.bz2

In [ ]:
!pip install python-telegram-bot==13.0

In [ ]:
import dlib
import io
import numpy as np
from PIL import Image
from telegram import InlineQueryResultArticle, InputTextMessageContent
from telegram.ext import Updater, CommandHandler
from telegram.ext import MessageHandler, Filters, InlineQueryHandler

In [ ]:
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')
face_recognition_model = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')

In [ ]:
TOKEN = '6171938814:AAGLQUEWO8apUUOfwiNSRuBIw2yBHFEeopU'

face_count = 0
embeddings = [0]

def similarity(embed1, embed2):
    return 1 - np.linalg.norm(embed1 - embed2)

def merge(img1, img2):
    h = int(np.round(np.sqrt(img1.size[0] * img2.size[0])))
    w = 2 * h

    img = Image.new("RGB", (w, h))
    img.paste(img1.resize((h, h)))
    img.paste(img2.resize((h, h)), (h, 0))

    return img

def start(update, context):
    global face_count
    global embeddings
    face_count = 0
    embeddings = [0]
    
    context.bot.send_message(chat_id=update.message.chat_id,
                             text="Hello, I am a bot, I will help you to calculate similarity of different faces.\n" \
                             "You can send me photos with people you want to recognize and use command '/compare a b' and I will tell how similar photos a and b are.")

def handle_photo(update, context):
    global face_count
    global embeddings
    message = update.message
    photo = message.photo[~0]
    with io.BytesIO() as fd:
        file_id = context.bot.get_file(photo.file_id)
        file_id.download(out=fd)
        fd.seek(0)

        image = Image.open(fd)
        image.load()
    
    faces = face_detector(np.array(image), 1)

    if len(faces) == 0:
        context.bot.send_message(chat_id=update.message.chat_id, text='No faces found')
        return
    context.bot.send_message(chat_id=update.message.chat_id, text='Found ' + str(len(faces)) + " face(s)")

    for f in faces:
        landmarks = shape_predictor(np.array(image), f)
        embedding = face_recognition_model.compute_face_descriptor(np.array(image), landmarks)
        embeddings.append(np.array(embedding))

        face_count += 1
        filename = "face"+str(face_count)+".jpg"
        image.crop([f.left(), f.top(), f.right(), f.bottom()]).save(filename)
        context.bot.send_photo(chat_id=update.message.chat_id, photo = open(filename, 'rb'), caption = "Face " + str(face_count))

def compare(update, context):
    global face_count
    global embeddings
    message = update.message
    words = message.text.split()

    if len(words) != 3:
        context.bot.send_message(chat_id=update.message.chat_id, text="Your request should look like '/compare 2 4'")
        return
    
    try:
        n1, n2 = int(words[1]), int(words[2])
    except:
        context.bot.send_message(chat_id=update.message.chat_id, text="It seems, your arguments are not integer numbers")
        return
    
    if (n1 < 1) or (n2 < 1) or (n1 > face_count) or (n2 > face_count):
        context.bot.send_message(chat_id=update.message.chat_id, text="It seems, your arguments are out of range, the amount of photos is " + str(face_count))
    else:
        sim = similarity(embeddings[n1], embeddings[n2])
        img1 = Image.open("face"+str(n1)+".jpg")
        img2 = Image.open("face"+str(n2)+".jpg")
        merge(img1, img2).save("comparison.jpg")
        context.bot.send_photo(chat_id=update.message.chat_id, photo = open("comparison.jpg", 'rb'),
                               caption = "Similarity of this photos is " + str(round(sim, 4)))


updater = Updater(token=TOKEN)
dispatcher = updater.dispatcher

photo_handler = MessageHandler(Filters.photo, handle_photo)
dispatcher.add_handler(photo_handler)

start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

compare_handler = CommandHandler('compare', compare)
dispatcher.add_handler(compare_handler)

updater.start_polling()
updater.idle()